# Jupyterで体験するアルゴリズムのかたち

## 概要

このノートブックは、Pythonの標準ライブラリとJupyterLabの基本機能だけを使って、
アルゴリズム・乱数・数理的構造・動き・パターンを視覚化するプログラムを集めたものである。

テキストだけで動きや構造を表現するもの（ASCIIアート風）と、
画像を表示するもの（SVGによる可視化）をそれぞれ収録している。
いずれも短いコードで実行でき、 **アルゴリズムの動作を「見て」「感じる」** ことを目的としている。

この資料ではコードの内容を理解することよりも，JupyterLabノートブックでどんなことができるかを体験することを重視している．
以下に示したコードセルを実行し（コードセルを選択してから実行（►）ボタンを押し），何が表示されるか試してみてほしい．

## print文

はじめは何のひねりもないprint文から実行してみる．これが動作しなかったら何かのエラーが起きていると推測される．

In [ ]:
print('Hello, JupyterLab!')

## テキスト花火

ランダムな位置に星印（＊）を表示して消すことで、花火のような動きを文字だけで再現する。
座標計算とクリア表示のしくみを使った簡単なアニメーションの例。

In [ ]:
import random, time, os
from IPython.display import clear_output

def text_fireworks(width=40, height=10, frames=30, delay=0.1):
    for _ in range(frames):
        clear_output(wait=True)
        # スパーク生成
        spark_x = random.randint(0, width-1)
        spark_y = random.randint(0, height-1)
        frame = []
        for y in range(height):
            row = ""
            for x in range(width):
                if abs(x-spark_x)+abs(y-spark_y) < random.randint(0,2):
                    row += "*"
                else:
                    row += " "
            frame.append(row)
        print("\n".join(frame))
        time.sleep(delay)

# 実行例：
text_fireworks()

## ASCII山脈

乱数によって高さを決め、#記号で山並みを描くプログラム。
数値データを文字で可視化する方法や、ノイズ・補間の考え方を学べる。

In [ ]:
import random

def ascii_mountain(width=80, height=20, peaks=5, roughness=0.4):
    heights = [random.random() for _ in range(peaks+1)]
    expanded = []
    for i in range(peaks):
        start, end = heights[i], heights[i+1]
        for t in range(width // peaks):
            p = t / (width//peaks)
            h = (1-p)*start + p*end + random.uniform(-roughness, roughness)
            expanded.append(h)
    max_h = max(expanded)
    for y in range(height, -1, -1):
        line = ""
        for h in expanded:
            line += "#" if h * height > y else " "
        print(line)

# 実行例：
ascii_mountain()

## テキスト版ライフゲーム

ライフゲームを文字だけで表現したバージョン。
「█」と空白だけで生命の誕生と消滅を観察でき、ルールが明確に見える。

このノートの後半にはグラフィック版を掲載している．

In [ ]:
import random, time, os
from IPython.display import clear_output

def text_life(width=30, height=15, steps=50, delay=0.1):
    grid = [[random.randint(0,1) for _ in range(width)] for _ in range(height)]

    def count8(x, y):
        s = 0
        for dy in (-1,0,1):
            for dx in (-1,0,1):
                if dx==0 and dy==0: continue
                s += grid[(y+dy)%height][(x+dx)%width]
        return s

    for _ in range(steps):
        clear_output(wait=True)
        for row in grid:
            print("".join("█" if c else " " for c in row))
        time.sleep(delay)
        grid = [[1 if (grid[y][x] and count8(x,y) in (2,3)) or (not grid[y][x] and count8(x,y)==3) else 0
                 for x in range(width)] for y in range(height)]

# 実行例：
text_life()

## 同心円のゆらぎアート

円の半径にランダムなゆらぎを与えて描くことで、手描き風の有機的な模様を生成する。
ループ・乱数・パラメータの影響を直感的に理解できるアート的プログラミングの例。

In [ ]:
# 同心円のゆらぎアート：ランダムで半径を揺らしたパス
from IPython.display import display, SVG
import random, math

def concentric_wobble(num=18, r0=20, step=10, cx=220, cy=160, jitter=4, seed=None):
    if seed is not None:
        random.seed(seed)
    w, h = 440, 320
    svg = [f'<svg xmlns="http://www.w3.org/2000/svg" width="{w}" height="{h}">']
    svg.append(f'<rect x="0" y="0" width="{w}" height="{h}" fill="#0b0b0f"/>')
    palette = ["#007acc","#ff5c8a","#ffd166","#06d6a0","#8338ec","#ef476f"]

    for k in range(num):
        r = r0 + k*step
        pts = []
        seg = max(40, int(2*math.pi*r/6))
        for i in range(seg+1):
            ang = 2*math.pi*i/seg
            rr = r + random.uniform(-jitter, jitter)
            x = cx + rr*math.cos(ang)
            y = cy + rr*math.sin(ang)
            pts.append((x,y))
        path = "M " + " L ".join(f"{x:.2f},{y:.2f}" for x,y in pts) + " Z"
        stroke = palette[k % len(palette)]
        svg.append(f'<path d="{path}" fill="none" stroke="{stroke}" stroke-opacity="0.75" stroke-width="2"/>')

    svg.append("</svg>")
    display(SVG("\n".join(svg)))

# 実行例：
concentric_wobble(num=22, r0=16, step=9, jitter=5, seed=2)


## リサージュ曲線

2つの異なる周波数・位相をもつ正弦波を組み合わせ、図形を描くプログラム。
単純な数式から複雑で美しいパターンが生まれる現象を通じて、周期・比・位相の概念を学べる。

In [ ]:
# リサージュ曲線：周波数比と位相で形が変わる
from IPython.display import display, SVG
import math

def lissajous_svg(a=3, b=4, delta=math.pi/2.5, A=150, B=110, cx=200, cy=160, samples=1500):
    w, h = 400, 320
    svg = [f'<svg xmlns="http://www.w3.org/2000/svg" width="{w}" height="{h}">']
    svg.append(f'<rect x="0" y="0" width="{w}" height="{h}" fill="#0b0b0f"/>')

    pts = []
    for i in range(samples):
        t = 2*math.pi * (i / (samples-1))
        x = cx + A * math.sin(a*t + delta)
        y = cy + B * math.sin(b*t)
        pts.append((x,y))
    poly = " ".join(f"{x:.2f},{y:.2f}" for x,y in pts)
    svg.append(f'<polyline points="{poly}" fill="none" stroke="#ffd166" stroke-width="2"/>')
    svg.append(f'<rect x="10" y="10" width="{w-20}" height="{h-20}" fill="none" stroke="#577590" stroke-width="1" opacity="0.6"/>')
    svg.append("</svg>")
    display(SVG("\n".join(svg)))

# 実行例：
lissajous_svg(a=5, b=7, delta=1.2)

## 迷路生成（深さ優先探索）

深さ優先探索（再帰的バックトラック法）で迷路を自動生成するプログラム。
アルゴリズムの探索順序が迷路構造として可視化され、ランダム性と規則性の両方を体感できる。

In [ ]:
# 迷路生成（DFS）: IPython.display で直接SVG描画
from IPython.display import display, SVG
import random

def maze_dfs(cols=31, rows=21, cell=20, seed=None):
    if seed is not None:
        random.seed(seed)
    # 1=壁, 0=道（奇数格子推奨）
    W, H = cols, rows
    grid = [[1]*W for _ in range(H)]

    def carve(x, y):
        grid[y][x] = 0
        dirs = [(2,0),(-2,0),(0,2),(0,-2)]
        random.shuffle(dirs)
        for dx, dy in dirs:
            nx, ny = x+dx, y+dy
            if 0 < nx < W-1 and 0 < ny < H-1 and grid[ny][nx] == 1:
                grid[y+dy//2][x+dx//2] = 0
                carve(nx, ny)

    carve(1, 1)

    bg = "#111"
    svg = [f'<svg xmlns="http://www.w3.org/2000/svg" width="{cols*cell}" height="{rows*cell}">']
    svg.append(f'<rect x="0" y="0" width="{cols*cell}" height="{rows*cell}" fill="{bg}"/>')
    for y in range(H):
        for x in range(W):
            if grid[y][x] == 1:
                svg.append(f'<rect x="{x*cell}" y="{y*cell}" width="{cell}" height="{cell}" fill="#222"/>')
    # スタート/ゴール
    svg.append(f'<rect x="{1*cell}" y="{1*cell}" width="{cell}" height="{cell}" fill="#06d6a0" opacity="0.7"/>')
    svg.append(f'<rect x="{(W-2)*cell}" y="{(H-2)*cell}" width="{cell}" height="{cell}" fill="#ef476f" opacity="0.7"/>')
    svg.append('</svg>')
    display(SVG("\n".join(svg)))

# 実行例：
maze_dfs(cols=31, rows=21, cell=18, seed=0)

## ウラムの渦（Ulam spiral）

自然数を渦巻き状に配置し、素数の位置だけを印として描く可視化。
素数分布に潜む秩序やパターンを、シンプルなループと条件分岐で表現する。

In [ ]:
# ウラムの渦：素数の位置にドットを打つ
from IPython.display import display, SVG

def ulam_spiral(n=101, dot=6, gap=1):
    if n % 2 == 0:
        n += 1
    size = n * (dot + gap) + gap
    cx = cy = size // 2

    def is_prime(k):
        if k < 2: return False
        if k % 2 == 0: return k == 2
        i, r = 3, int(k**0.5)
        while i <= r:
            if k % i == 0: return False
            i += 2
        return True

    svg = [f'<svg xmlns="http://www.w3.org/2000/svg" width="{size}" height="{size}">']
    svg.append(f'<rect x="0" y="0" width="{size}" height="{size}" fill="#0b0b0f"/>')
    colors = ["#ffffff", "#ffd166", "#06d6a0", "#ef476f", "#8ecae6"]

    x = y = 0
    dx, dy = 1, 0
    step = 1
    steps_in_dir = 0
    change_after = 1
    change_count = 0

    num = 1
    for _ in range(n*n):
        if is_prime(num):
            px = cx + x*(dot+gap)
            py = cy + y*(dot+gap)
            color = colors[num % len(colors)]
            svg.append(f'<rect x="{px}" y="{py}" width="{dot}" height="{dot}" fill="{color}"/>')
        # 進行
        x += dx; y += dy
        steps_in_dir += 1
        if steps_in_dir == step:
            steps_in_dir = 0
            dx, dy = -dy, dx
            change_count += 1
            if change_count == change_after:
                change_count = 0
                change_after = 2
                step += 1
        num += 1

    svg.append(f'<rect x="0" y="0" width="{size}" height="{size}" fill="none" stroke="#223" stroke-width="1"/>')
    svg.append('</svg>')
    display(SVG("\n".join(svg)))

# 実行例：
ulam_spiral(n=111, dot=5, gap=1)

## マンデルブロ集合

複素平面上の点に対して発散判定を繰り返し、結果を色分けして描くプログラム。
簡潔な反復式から生まれる無限に複雑な構造を通じて、再帰的思考や数値計算の面白さを学べる。

このコードを実行すると待ち時間が他のコードより長いので注意せよ．

In [ ]:
# マンデルブロ集合：単純な矩形塗り SVG
from IPython.display import display, SVG

def mandelbrot_svg(w=360, h=240, max_iter=60, x_min=-2.5, x_max=1.0, y_min=-1.2, y_max=1.2):
    def color(i):
        if i >= max_iter:
            return "#0b0b0f"
        t = i / max_iter
        r = int(255 * (0.3 + 0.7 * t))
        g = int(255 * (t**0.5))
        b = int(255 * (1.0 - t) * 0.8)
        return f"rgb({r},{g},{b})"

    svg = [f'<svg xmlns="http://www.w3.org/2000/svg" width="{w}" height="{h}">']
    for py in range(h):
        cy = y_min + (y_max - y_min) * (py / (h - 1))
        row = []
        for px in range(w):
            cx = x_min + (x_max - x_min) * (px / (w - 1))
            x = y = 0.0
            i = 0
            while x*x + y*y <= 4.0 and i < max_iter:
                x, y = x*x - y*y + cx, 2*x*y + cy
                i += 1
            row.append(f'<rect x="{px}" y="{py}" width="1" height="1" fill="{color(i)}"/>')
        svg.append("".join(row))
    svg.append("</svg>")
    display(SVG("\n".join(svg)))

# 実行例：
mandelbrot_svg(w=420, h=300, max_iter=80)

## ライフゲーム（Game of Life）

「生」「死」のルールをもつセルの集合が自動的に変化していくシミュレーション。
生命現象や秩序形成のモデルとして知られるセルオートマトンの基本を体験できる。

In [ ]:
# ライフゲーム：SVGを都度描画し clear_output(wait=True) で更新
from IPython.display import display, SVG, clear_output
import random, time

def game_of_life(width=60, height=40, density=0.18, steps=120, cell=10, delay=0.05, seed=None):
    if seed is not None:
        random.seed(seed)
    grid = [[1 if random.random() < density else 0 for _ in range(width)] for _ in range(height)]

    def count8(x, y):
        s = 0
        for dy in (-1,0,1):
            for dx in (-1,0,1):
                if dx==0 and dy==0: 
                    continue
                nx = (x+dx) % width
                ny = (y+dy) % height
                s += grid[ny][nx]
        return s

    for gen in range(steps):
        # 描画
        w, h = width*cell, height*cell
        svg = [f'<svg xmlns="http://www.w3.org/2000/svg" width="{w}" height="{h}">']
        svg.append(f'<rect x="0" y="0" width="{w}" height="{h}" fill="#0b0b0f"/>')
        alive_color = "#00c2a8"
        for y in range(height):
            row = grid[y]
            for x in range(width):
                if row[x]:
                    svg.append(f'<rect x="{x*cell}" y="{y*cell}" width="{cell}" height="{cell}" fill="{alive_color}"/>')
        svg.append(f'<text x="6" y="{h-6}" fill="#8ac926" font-size="12" font-family="monospace">Gen {gen}</text>')
        svg.append("</svg>")
        clear_output(wait=True)
        display(SVG("\n".join(svg)))

        # 次世代
        newg = [[0]*width for _ in range(height)]
        for y in range(height):
            for x in range(width):
                c = grid[y][x]
                n = count8(x,y)
                if c and n in (2,3):
                    newg[y][x] = 1
                elif (not c) and n == 3:
                    newg[y][x] = 1
        grid = newg
        time.sleep(delay)

# 実行例：
game_of_life(width=50, height=30, cell=12, delay=0.06, seed=1)

## 課題（なし）

このノートブックには課題は設定されていません．